In [16]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

df2 = pd.read_csv(r"C:\Users\Sooraj Suresh\OneDrive\Documents\Desktop\result.csv")
makeup = pd.read_csv(r"C:\Users\Sooraj Suresh\OneDrive\Documents\Desktop\makeup_final.csv")
entries = len(df2)
LABELS = list(df2.label.unique())

# features

features = ['normal', 'dry', 'oily', 'combination', 'acne', 'sensitive', 'fine lines', 'wrinkles', 'redness',
            'dull', 'pore', 'pigmentation', 'blackheads', 'whiteheads', 'blemishes', 'dark circles', 'eye bags', 'dark spots']


# utility functions

def search_concern(target, i):
    if target in df2.iloc[i]['concern']:
        return True
    return False


def name2index(name):
    return df2[df2["name"] == name].index.tolist()[0]


def index2prod(index):
    return df2.iloc[index]


def wrap(info_arr):
    result = {}
#     print(info_arr)
    result['brand'] = info_arr[0]
    result['name'] = info_arr[1]
    result['url'] = info_arr[2]
    result['img'] = info_arr[3]
    result['skin type'] = info_arr[4]
    result['concern'] = str(info_arr[5]).split(',')
    return result

def wrap_makeup(info_arr):
    result = {}
#     print(info_arr)
    result['brand'] = info_arr[0]
    result['name'] = info_arr[1]
    result['url'] = info_arr[2]
    result['img'] = info_arr[3]
    result['skin type'] = info_arr[4]
    result['skin tone'] = info_arr[5]
    return result

one_hot_encodings = np.zeros([entries, len(features)])


#skin types first
for i in range(entries):
    for j in range(5):
        target = features[j]
        sk_type = df2.iloc[i]['skin type']
        if sk_type == 'all':
            one_hot_encodings[i][0:5] = 1
        elif target == sk_type:
            one_hot_encodings[i][j] = 1

#other features
# Ensure that 'concern' is a string before checking for feature
for i in range(len(df2)):
    for j in range(5, len(features)):
        feature = features[j]
        if isinstance(df2.iloc[i]['concern'], str) and feature in df2.iloc[i]['concern']:
            one_hot_encodings[i][j] = 1


# recommend top 5 similar items from a category


def recs_cs(vector = None, name = None, label = None, count = 5):
    products = []
    if name:
        idx = name2index(name)
        fv = one_hot_encodings[idx]
    elif vector:
        fv = vector
    cs_values = cosine_similarity(np.array([fv, ]), one_hot_encodings)
    df2['cs'] = cs_values[0]
    
    if label:
        dff = df2[df2['label'] == label]
    else:
        dff = df2
    
    if name:
        dff = dff[dff['name'] != name]
    recommendations = dff.sort_values('cs', ascending=False).head(count)
    #   print(f"Top {count} matching {label} items")
    data = recommendations[['brand', 'name', 'price', 'url','img','skin type','concern']].to_dict('split')['data']
    for element in data:
        products.append(wrap(element))
    return products

    # overall recommendation


def recs_essentials(vector = None, name = None):
#     print("ESSENTIALS:")
    response = {}
    for label in LABELS:
#         print(f"{label}:")
        if name: 
            r = recs_cs(None, name, label)
        elif vector:
            r = recs_cs(vector, None, label)
        response[label] = r
    return response



def makeup_recommendation(skin_tone, skin_type):
    result = []
    dff = pd.DataFrame()
    dff = dff.append(makeup[(makeup['skin tone'] == skin_tone) & (makeup['skin type'] == skin_type) & (makeup['label'] == 'foundation')].head(2))
    dff = dff.append(makeup[(makeup['skin tone'] == skin_tone) & (makeup['skin type'] == skin_type) & (makeup['label'] == 'concealer')].head(2))
    dff = dff.append(makeup[(makeup['skin tone'] == skin_tone) & (makeup['skin type'] == skin_type) & (makeup['label'] == 'primer')].head(2))
    dff= dff.sample(frac = 1)
    data = dff[['brand', 'name', 'url', 'img', 'skin type', 'skin tone']].to_dict('split')['data']
    for element in data:
        result.append(wrap_makeup(element))
    return result

In [17]:
import pandas as pd

# Load the web scraped data (replace with actual CSV file path)
dff = pd.read_csv(r"C:\Users\Sooraj Suresh\OneDrive\Documents\Desktop\result.csv")

# Define mapping of acne subtypes to ingredients
acne_subtype_to_ingredients = {
    'blackheads': ['salicylic acid', 'retinoids', 'benzoyl peroxide'],
    'whiteheads': ['salicylic acid', 'retinoids', 'AHA', 'BHA'],
    'cysts': ['benzoyl peroxide', 'sulfur', 'retinoids'],
    'papules': ['benzoyl peroxide', 'salicylic acid', 'sulfur'],
    'pustules': ['benzoyl peroxide', 'salicylic acid', 'sulfur', 'antibiotics']
}

acne_type = 'blackheads'

# Function to recommend products based on acne concern
def recommend_products_based_on_acne_type(acne_type, dff):
    # Get the list of ingredients for the acne type
    ingredients_needed = acne_subtype_to_ingredients.get(acne_type.lower())
    
    if not ingredients_needed:
        return "Invalid acne type"
    
    # Filter products that contain any of the required ingredients
    recommendations = []
    for _, row in dff.iterrows():
        # Check if the ingredients column contains any of the relevant ingredients for the acne type
        key_ingredient = row['key ingredient']
        
        if isinstance(key_ingredient, str): 
            if any(ingredient in row['key ingredient'].lower() for ingredient in ingredients_needed):
                recommendations.append(row)
    
    # If no recommendations are found
    if not recommendations:
        return "No products found for this acne type."
    
    # Convert recommendations to DataFrame
    recommended_df = pd.DataFrame(recommendations)
    
    # Extract the necessary columns for display
    recommended_df = recommended_df[['brand', 'name', 'url', 'skin type', 'concern', 'key ingredient']]
    
    return recommended_df

# Example: Recommend products for 'blackheads'

recommended_products = recommend_products_based_on_acne_type(acne_type, dff)

# Display the recommended products
print(recommended_products)


Invalid acne type
